In [2]:
!nvidia-smi

Tue May  9 10:44:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.40       Driver Version: 517.40       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   36C    P8    11W /  N/A |   2471MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!python -m pip install -q --user diffusers==0.14.0 transformers xformers git+https://github.com/huggingface/accelerate.git
!python -m pip install -q opencv-contrib-python
!python -m pip install -q controlnet_aux

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.13.1 requires torch==1.13.1, but you have torch 2.0.0 which is incompatible.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 2.0.0 which is incompatible.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 2.0.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.19 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.


In [9]:
# import
from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image

input_path = "ramen.JPG"

# 画像読み込み
image = load_image(input_path)
image


ImportError: cannot import name 'StableDiffusionControlNetPipeline' from 'diffusers' (c:\Users\honda\AppData\Local\Programs\Python\Python310\lib\site-packages\diffusers\__init__.py)

In [ ]:
import cv2
from PIL import Image
import numpy as np

image = np.array(image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)
canny_image

In [ ]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import UniPCMultistepScheduler

# Canny特徴用のControlNetモデルとrunwaylml/stable-diffusion-v1-5をロード
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)

# Stable DiffusionのデフォルトのPNDMSchedulerを使う代わりに、UniPCMultistepSchedulerという現在最も高速な拡散モデルスケジューラの1つを使用
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# パイプラインを直接GPUにロードする代わりに、enable_model_cpu_offload関数でCPUオフロードを有効にする
pipe.enable_model_cpu_offload()

# FlashAttention/xformersのアテンションレイヤーアクセラレーションを有効にする
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

# prompt = ", best quality, extremely detailed"
# prompt = [t + prompt for t in ["spring", "summer", "autumn", "winter"]]
prompt = ["pasta, best quality, extremely detailed"]
generator = [torch.Generator(device="cpu").manual_seed(2) for i in range(len(prompt))]

output = pipe(
    prompt,
    canny_image,
    negative_prompt=["monochrome, lowres, bad anatomy, worst quality, low quality"] * len(prompt),
    generator=generator,
    num_inference_steps=20,
)

output.images[0]